In [1]:
import pandas as pd 
import numpy as np

In [2]:

name_distances = ['left_shoulder_left_wrist',
                'right_shoulder_right_wrist',
                'left_hip_left_ankle',
                'right_hip_right_ankle',
                'left_hip_left_wrist',
                'right_hip_right_wrist',
                'left_shoulder_left_ankle',
                'right_shoulder_right_ankle',
                'left_hip_right_wrist',
                'right_hip_left_wrist',
                'left_elbow_right_elbow',
                'left_knee_right_knee',
                'left_wrist_right_wrist',
                'left_ankle_right_ankle',
                'left_hip_avg_left_wrist_left_ankle',
                'right_hip_avg_right_wrist_right_ankle']

name_angles = ['right_elbow_right_shoulder_right_hip',
            'left_elbow_left_shoulder_left_hip',
            'right_knee_mid_hip_left_knee',
            'right_hip_right_knee_right_ankle',
            'left_hip_left_knee_left_ankle',
            'right_wrist_right_elbow_right_shoulder',
            'left_wrist_left_elbow_left_shoulder']

def cal_distances(point_1, point_2):
    # [x1_coords, y1_coords, z1_coords] = [ -5.889507, -57.63752, -45.01975]
    # [x2_coords, y2_coords, z2_coords] = [ -4.656085, -62.832863, -44.571823]
    p1 = np.array([point_1[0], point_1[1], point_1[2]])
    p2 = np.array([point_2[0], point_2[1], point_2[2]])

    squared_dist = np.sum((p1-p2)**2, axis=0)
    dist = np.sqrt(squared_dist)
    return dist

def create_dist_Series(dist_between_string):
    ### split string
    index = dist_between_string.find("_", int(len(dist_between_string)/2 - 2), -1) ### find underscore in mid string
    A_point = dist_between_string[:index]
    B_point = dist_between_string[index+1:]

    # Define the function
    def norm_2(point_1, point_2):
        A = [point_1[0], point_1[1], point_1[2]]
        B = [point_2[0], point_2[1], point_2[2]]
        dist = cal_distances(point_1= A, point_2= B)
        return dist

    # Apply the function to the DataFrame
    out_distances = df_landmarks.apply(lambda row: norm_2(row[['x_'+A_point, 'y_'+A_point, 'z_'+A_point]], row[['x_'+B_point, 'y_'+B_point, 'z_'+B_point]]), axis=1)
    return out_distances

def create_avg_dist(avg_between_string):
    index = avg_between_string.find("avg_") ### find avg_ in mid string
    A_point = avg_between_string[:index-1]
    other = avg_between_string[index+4:]

    sub_str = other
    index = sub_str.find("_", int(len(sub_str)/2 - 2), -1) ### find underscore in mid string
    B_point = sub_str[:index]
    C_point = sub_str[index+1:]
    def avg_(row):
        return (row[A_point+ '_' + B_point] + row[A_point+ '_' + C_point]) /2

    # Apply the function to the DataFrame
    out_distances = df_out_distances.apply(lambda row: avg_(row), axis= 1)
    return out_distances


In [ ]:
class Landmarks_2_Features():
    def __init__(self, df_landmarks):
        
        """
            landmarks is list 33 points with x, y, z -> len(landmarks) == 99
        """
        self.df_landmarks = df_landmarks
        self.df_out_distances = pd.DataFrame()
        self.features_3d_distances = name_distances
        self.features_angles = name_angles

    def to_3d_distances(self, landmarks):
        self.df_landmarks = landmarks
        def cal_distances(point_1, point_2):
            # [x1_coords, y1_coords, z1_coords] = [ -5.889507, -57.63752, -45.01975]
            # [x2_coords, y2_coords, z2_coords] = [ -4.656085, -62.832863, -44.571823]
            p1 = np.array([point_1[0], point_1[1], point_1[2]])
            p2 = np.array([point_2[0], point_2[1], point_2[2]])

            squared_dist = np.sum((p1-p2)**2, axis=0)
            dist = np.sqrt(squared_dist)
            return dist

        def create_dist_Series(dist_between_string):
            ### split string
            index = dist_between_string.find("_", int(len(dist_between_string)/2 - 2), -1) ### find underscore in mid string
            A_point = dist_between_string[:index]
            B_point = dist_between_string[index+1:]

            # Define the function
            def norm_2(point_1, point_2):
                A = [point_1[0], point_1[1], point_1[2]]
                B = [point_2[0], point_2[1], point_2[2]]
                dist = cal_distances(point_1= A, point_2= B)
                return dist

            # Apply the function to the DataFrame
            out_distances = self.df_landmarks.apply(lambda row: norm_2(row[['x_'+A_point, 'y_'+A_point, 'z_'+A_point]], row[['x_'+B_point, 'y_'+B_point, 'z_'+B_point]]), axis=1)
            return out_distances

        def create_avg_dist(avg_between_string):
            index = avg_between_string.find("avg_") ### find avg_ in mid string
            A_point = avg_between_string[:index-1]
            other = avg_between_string[index+4:]

            sub_str = other
            index = sub_str.find("_", int(len(sub_str)/2 - 2), -1) ### find underscore in mid string
            B_point = sub_str[:index]
            C_point = sub_str[index+1:]
            def avg_(row):
                return (row[A_point+ '_' + B_point] + row[A_point+ '_' + C_point]) /2

            # Apply the function to the DataFrame
            out_distances = self.df_out_distances.apply(lambda row: avg_(row), axis= 1)
            return out_distances

        for i in  range(len(self.features_3d_distances)):
            index = self.features_3d_distances[i].find("avg_")
            if index == -1:
                self.df_out_distances[self.features_3d_distances[i]] = create_dist_Series(self.features_3d_distances[i])
            else:
                self.df_out_distances[self.features_3d_distances[i]] = create_avg_dist(self.features_3d_distances[i])
                pass
        
    def to_angles(self):

        pass

# demo

In [4]:
path_demo = 'dataframes_demo.csv'
df_landmarks = pd.read_csv(path_demo)
df_landmarks.drop(index=[0, 1], axis=0, inplace=True)

instance_1 = [i for i in range(99)]
# xử lý -> instance_1
df_landmarks.loc[0] = instance_1

In [5]:
import numpy as np

def cal_distances(point_1, point_2):
    # [x1_coords, y1_coords, z1_coords] = [ -5.889507, -57.63752, -45.01975]
    # [x2_coords, y2_coords, z2_coords] = [ -4.656085, -62.832863, -44.571823]
    p1 = np.array([point_1[0], point_1[1], point_1[2]])
    p2 = np.array([point_2[0], point_2[1], point_2[2]])

    squared_dist = np.sum((p1-p2)**2, axis=0)
    dist = np.sqrt(squared_dist)
    return dist

In [11]:
### get list of columns : df_distances

def create_dist_Series(dist_between_string):
    ### split string
    index = dist_between_string.find("_", int(len(dist_between_string)/2 - 2), -1) ### find underscore in mid string
    A_point = dist_between_string[:index]
    B_point = dist_between_string[index+1:]

    # Define the function
    def norm_2(point_1, point_2):
        A = [point_1[0], point_1[1], point_1[2]]
        B = [point_2[0], point_2[1], point_2[2]]
        dist = cal_distances(point_1= A, point_2= B)
        return dist

    # Apply the function to the DataFrame
    out_distances = df_landmarks.apply(lambda row: norm_2(row[['x_'+A_point, 'y_'+A_point, 'z_'+A_point]], row[['x_'+B_point, 'y_'+B_point, 'z_'+B_point]]), axis=1)
    return out_distances

def create_avg_dist(avg_between_string):
    index = avg_between_string.find("avg_") ### find avg_ in mid string
    A_point = avg_between_string[:index-1]
    other = avg_between_string[index+4:]

    sub_str = other
    index = sub_str.find("_", int(len(sub_str)/2 - 2), -1) ### find underscore in mid string
    B_point = sub_str[:index]
    C_point = sub_str[index+1:]
    def avg_(row):
        return (row[A_point+ '_' + B_point] + row[A_point+ '_' + C_point]) /2

    # Apply the function to the DataFrame
    out_distances = df_out_distances.apply(lambda row: avg_(row), axis= 1)
    return out_distances

# Importing Pandas to create DataFrame
import pandas as pd
  
# Creating Empty DataFrame and Storing it in variable df
df_out_distances = pd.DataFrame()

# name_distances = list(df_distances.columns)

name_distances = ['left_shoulder_left_wrist',
                'right_shoulder_right_wrist',
                'left_hip_left_ankle',
                'right_hip_right_ankle',
                'left_hip_left_wrist',
                'right_hip_right_wrist',
                'left_shoulder_left_ankle',
                'right_shoulder_right_ankle',
                'left_hip_right_wrist',
                'right_hip_left_wrist',
                'left_elbow_right_elbow',
                'left_knee_right_knee',
                'left_wrist_right_wrist',
                'left_ankle_right_ankle',
                'left_hip_avg_left_wrist_left_ankle',
                'right_hip_avg_right_wrist_right_ankle']

for i in  range(len(name_distances)):
    index = name_distances[i].find("avg_")
    if index == -1:
        df_out_distances[name_distances[i]] = create_dist_Series(name_distances[i])
    else:
        df_out_distances[name_distances[i]] = create_avg_dist(name_distances[i])


In [12]:
X = df_out_distances.iloc[0].to_numpy()
X = X.reshape((1,-1))

In [13]:
X

array([[20.78460969, 20.78460969, 20.78460969, 20.78460969, 41.56921938,
        41.56921938, 83.13843876, 83.13843876, 36.37306696, 46.7653718 ,
         5.19615242,  5.19615242,  5.19615242,  5.19615242, 31.17691454,
        31.17691454]])